In [3]:
from toloka.interface.pool import Pool
from toloka.interface.toloka import Toloka

async with Toloka.init_session():
    pool = Pool()
    #await pool.get_data()

In [4]:
from pprint import pprint
pprint(pool.data.to_dict())

AttributeError: 'NoneType' object has no attribute 'to_dict'

In [10]:
Toloka.init_session('main')

In [13]:
Pool.token

'AgAAAAA53AM2AAIbuvg9CjBknkT0p99g-Xw_Ccs'